In [31]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [3]:
class CNN(nn.Module):
    def __init__(self, in_channels=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.layer1 = nn.Linear(16*7*7, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.layer1(x)
        return x
    

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

learning_rate = 1e-3
in_channels = 1
num_classes = 10
batch_size = 64
num_epochs = 2

In [8]:
train_dataset = datasets.MNIST(root='Data/MNIST-Dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
model = CNN(in_channels=in_channels, num_classes=num_classes)
model.to(device)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer1): Linear(in_features=784, out_features=10, bias=True)
)

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
writer = SummaryWriter(f'runs/MNIST/tensorboard_tutorial')

In [ ]:
batch_sizes = [1, 64, 128, 1024]
learning_rates = [1e-1, 1e-2, 1e-3, 1e-4]

In [17]:
step = 0
for epoch in range(num_epochs):
    losses = []
    accuracies = []
    loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for batch_idx, (data, targets) in loop_info:
        data = data.to(device)
        targets = targets.to(device)
        
        predicted_output = model(data)
        loss = loss_fn(predicted_output, targets)
        losses.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Calculating accuracy
        _, predictions = predicted_output.max(1)
        num_correct = (predictions == targets).sum()
        running_acc = float(num_correct) / float(data.shape[0])
        
        # Writing to tensorboard
        writer.add_scalar('Training Loss', loss, global_step=step)
        writer.add_scalar('Training Accuracy', running_acc, global_step=step)
        step += 1
        
        loop_info.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop_info.set_postfix(acc=running_acc, loss=loss)
print("Training Completed!!")

In [19]:
# Run and see results on tensorflow.
# In anaconda cmd line, in current folder
# tensorboard --logdir --runs

In [23]:
# Experiment 1
# Visualization 1
# Graphs for Loss and Acc for diff batch_size and LR

num_epochs = 1
batch_sizes = [64, 1024]
learning_rates = [1e-2, 1e-3, 1e-4]
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        # Just to reset params
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        writer = SummaryWriter(f'runs/MNIST/experiment/BatchSize{batch_size} LR{learning_rate}')
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
            for batch_idx, (data, targets) in loop_info:
                data = data.to(device)
                targets = targets.to(device)

                predicted_output = model(data)
                loss = loss_fn(predicted_output, targets)
                losses.append(loss)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculating accuracy
                _, predictions = predicted_output.max(1)
                num_correct = (predictions == targets).sum()
                running_acc = float(num_correct) / float(data.shape[0])

                # Writing to tensorboard
                writer.add_scalar('Training Loss', loss, global_step=step)
                writer.add_scalar('Training Accuracy', running_acc, global_step=step)
                step += 1

                loop_info.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
                loop_info.set_postfix(acc=running_acc, loss=loss)
print("Training Completed!!")

Training Completed!!


In [26]:
# Experiment 2 
# Visualization 2
# Visualization for HPARAMS

num_epochs = 1
batch_sizes = [64, 1024]
learning_rates = [1e-2, 1e-3, 1e-4]
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        # Just to reset params
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        writer = SummaryWriter(f'runs/MNIST/experiment/BatchSize{batch_size} LR{learning_rate}')
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
            for batch_idx, (data, targets) in loop_info:
                data = data.to(device)
                targets = targets.to(device)

                predicted_output = model(data)
                loss = loss_fn(predicted_output, targets)
                losses.append(loss)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculating accuracy
                _, predictions = predicted_output.max(1)
                num_correct = (predictions == targets).sum()
                running_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_acc)
                
                # Writing to tensorboard
                # Showing Training Loss and Accuracy over time
                writer.add_scalar('Training Loss', loss, global_step=step)
                writer.add_scalar('Training Accuracy', running_acc, global_step=step)
                step += 1

                loop_info.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
                loop_info.set_postfix(acc=running_acc, loss=loss)
            
            # For Showing HPARAMS
            writer.add_hparams(
                hparam_dict = {
                'lr': learning_rate,
                'batch_size': batch_size
                },
                metric_dict = {
                'accuracy': sum(accuracies) / len(accuracies),
                'loss': sum(losses) / len(losses)
                })
print("Training Completed!!")

Training Completed!!


In [27]:
# Experiment 2 
# Visualization 2
# Visualization for HPARAMS

num_epochs = 1
batch_sizes = [64]
learning_rates = [1e-4]
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        # Just to reset params
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        writer = SummaryWriter(f'runs/MNIST/experiment-viz/BatchSize{batch_size} LR{learning_rate}')
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
            for batch_idx, (data, targets) in loop_info:
                data = data.to(device)
                targets = targets.to(device)

                predicted_output = model(data)
                loss = loss_fn(predicted_output, targets)
                losses.append(loss)

                # Showing image of each epoch
                img_grid = torchvision.utils.make_grid(data)
                writer.add_image('mninst_images', img_grid)
                
                # Seeing weights distribution of each batch
                writer.add_histogram('layer1', model.layer1.weight)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculating accuracy
                _, predictions = predicted_output.max(1)
                num_correct = (predictions == targets).sum()
                running_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_acc)
                
                # Writing to tensorboard
                # Showing Training Loss and Accuracy over time
                writer.add_scalar('Training Loss', loss, global_step=step)
                writer.add_scalar('Training Accuracy', running_acc, global_step=step)
                step += 1

                loop_info.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
                loop_info.set_postfix(acc=running_acc, loss=loss)
            
            # For Showing HPARAMS
            writer.add_hparams(
                hparam_dict = {
                'lr': learning_rate,
                'batch_size': batch_size
                },
                metric_dict = {
                'accuracy': sum(accuracies) / len(accuracies),
                'loss': sum(losses) / len(losses)
                })
print("Training Completed!!")

Training Completed!!


In [32]:
# Tensorboard embedding projection
# To see how does the model evolve during training

num_epochs = 1
batch_sizes = [256]
learning_rates = [1e-4]
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        step = 0
        # Just to reset params
        model = CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device)
        writer = SummaryWriter(f'runs/MNIST/experiment-viz/BatchSize{batch_size} LR{learning_rate}')
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0)
        train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            losses = []
            accuracies = []
            loop_info = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
            for batch_idx, (data, targets) in loop_info:
                data = data.to(device)
                targets = targets.to(device)

                predicted_output = model(data)
                loss = loss_fn(predicted_output, targets)
                losses.append(loss)

                
                # Showing image of each epoch
                img_grid = torchvision.utils.make_grid(data)
                writer.add_image('mninst_images', img_grid)
                
                # Seeing weights distribution of each batch
                writer.add_histogram('layer1', model.layer1.weight)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Calculating accuracy
                _, predictions = predicted_output.max(1)
                num_correct = (predictions == targets).sum()
                running_acc = float(num_correct) / float(data.shape[0])
                accuracies.append(running_acc)
                
                 # Evolution during training
                features = data.reshape(data.shape[0], -1)
                class_labels = [classes[label] for label in predictions]
                if (batch_idx==230):
                    writer.add_embedding(features, metadata=class_labels, label_img=data)
                
                
                # Writing to tensorboard
                # Showing Training Loss and Accuracy over time
                writer.add_scalar('Training Loss', loss, global_step=step)
                writer.add_scalar('Training Accuracy', running_acc, global_step=step)
                step += 1

                loop_info.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
                loop_info.set_postfix(acc=running_acc, loss=loss)
            
            # For Showing HPARAMS
            writer.add_hparams(
                hparam_dict = {
                'lr': learning_rate,
                'batch_size': batch_size
                },
                metric_dict = {
                'accuracy': sum(accuracies) / len(accuracies),
                'loss': sum(losses) / len(losses)
                })
print("Training Completed!!")

Training Completed!!
